## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24
country = "Denmark"; # Possible options: Portugal, Slovenia, Greece, Denmark, this if apply only to Slovenia and Denmark, because both countries have biomass

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 19:02:39


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.Pl = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.Php = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]))
model.Pb = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,4]))
model.Pimp = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,5]))
model.Ppv = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,6]))
model.Pwind = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,7]))
model.Phid = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,8]))
model.Pbio = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Pgen = pyo.Var(model.t, domain =  pyo.NonNegativeReals, initialize = 0)
model.Pres = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Pex = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _RES_balance(m,t):
    return m.Pres[t] == m.Pwind[t] + m.Ppv[t] + m.Phid[t] + m.Pbio[t]
model.RES_balance = pyo.Constraint(model.t, rule = _RES_balance)

def _total_balance(m,t):
    #if (country == "Slovenia"):
        return m.Pgen[t] + m.Pres[t] + m.Pimp[t] - m.Pex[t] == m.Pl[t] + m.Pb[t] +sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
    #else:
        #return m.Pgen[t] + m.Pres[t] + m.Pimp[t] - m.Pex[t] ==  m.Pl[t] + m.Pb[t] + sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
model.total_balance = pyo.Constraint(model.t, rule = _total_balance)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
    #return m.EEVmin[ev] <= m.EEV[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)


## Objective function

In [11]:
def _FOag(m):
    return sum(m.Pgen[t] for t in np.arange(1, n_time + 1)) + sum([m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmplxmbhwse
  Lower bound: 679149730.5588425
  Upper bound: 679149730.5588425
  Number of objectives: 1
  Number of constraints: 917754
  Number of variables: 759553
  Number of nonzeros: 1829178
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 2.95
  Termination condition: optimal
  Termination message: Primal simplex - Optimal\x3a Objective = 6.7914973056e+08
  Error rc: 0
  Time: 6.72814154624939
# ----------------------------------------------------------
#   Solution Information
# -------------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

679149730.5588424

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 19:05:50
Dif: 0:03:11


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))

Charge cost: 11667100.604484418


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy()/1000000)))

Total Charge: 60.16329651767804


In [18]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,...,61355.454847,61355.454847,61355.454847,68004.545152,68004.545152,68004.545152,68004.545152,68004.545152,68004.545152,64680.0
2,43914.400342,43914.400342,43914.400342,43914.400342,43914.400342,43914.400342,43914.400342,43914.400342,43914.400342,43914.400342,...,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.0
3,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,...,8105.634469,8105.634469,8105.634469,8105.634469,8105.634469,8105.634469,8105.634469,8105.634469,8105.634469,8100.0
4,8100.000000,8100.000000,8100.000000,8133.806812,8133.806812,8133.806812,8133.806812,8133.806812,8133.806812,8133.806812,...,8116.903406,8116.903406,8116.903406,8116.903406,8116.903406,8116.903406,8116.903406,8116.903406,8116.903406,8100.0
5,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,...,36733.917045,34700.875567,34700.875567,34700.875567,34700.875567,48932.165910,46899.124433,44866.082955,42833.041478,40800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6325,44400.000000,44400.000000,44400.000000,44400.000000,44400.000000,44400.000000,44400.000000,44400.000000,44400.000000,44400.000000,...,57025.472631,52888.208947,48750.945262,44613.681578,44613.681578,40476.417893,36339.154209,36339.154209,32201.890524,44400.0
6326,46800.000000,46800.000000,46800.000000,48405.030549,48405.030549,48405.030549,48405.030549,48405.030549,48405.030549,48405.030549,...,47602.515275,47602.515275,47602.515275,47602.515275,47602.515275,47602.515275,47602.515275,47602.515275,47602.515275,46800.0
6327,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,...,59422.821494,59422.821494,59422.821494,59422.821494,59422.821494,59422.821494,59422.821494,59422.821494,59422.821494,57000.0
6328,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,...,65676.972600,65676.972600,65676.972600,65676.972600,65676.972600,65676.972600,65676.972600,65676.972600,65676.972600,64680.0


In [19]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,6999.042426,0.0,0.000000,0.0,0.0,0.0,0.000000
2,6556.6323,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,11.862039,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0000,0.0,0.0,35.586117,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
5,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,17120.349284,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6325,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,17195.129642
6326,0.0000,0.0,0.0,1689.505841,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
6327,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,5100.676829,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
6328,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,5101.110316


In [20]:
PEV_df.sum()

1     4.363414e+06
2     4.027367e+06
3     3.049860e+06
4     3.503841e+06
5     3.471848e+06
6     2.802645e+05
7     1.394803e+05
8     3.355881e+06
9     2.182627e+05
10    1.614589e+06
11    2.627893e+06
12    2.380489e+05
13    2.193871e+06
14    1.466228e+06
15    3.137412e+06
16    6.961429e+05
17    2.134768e+06
18    3.097060e+06
19    3.307345e+06
20    3.396911e+06
21    3.082221e+06
22    2.146500e+06
23    2.726146e+06
24    5.887941e+06
dtype: float64

In [21]:
Pgen_df = ext_pyomo_vals(model.Pgen)

Pgen_df

,0
1,9.376070e+06
2,9.063047e+06
3,7.857392e+06
4,8.041461e+06
5,7.893129e+06
6,5.023961e+06
7,9.839294e+05
8,4.200669e+06
9,1.236724e+06
10,2.637004e+06


In [22]:
Pres_df = ext_pyomo_vals(model.Pres)

Pres_df

,0
1,5.202029e+06
2,5.864105e+06
3,5.383119e+06
4,4.117823e+06
5,3.434997e+06
6,4.004192e+06
7,5.191122e+06
8,4.282210e+06
9,4.371468e+06
10,3.906558e+06


In [23]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [24]:
Etriprelax_df.sum()

1          0.000000
2          0.000000
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11         0.000000
12         0.000000
13         0.000000
14         0.000000
15      1352.438457
16         0.000000
17     12097.491539
18         0.000000
19         0.000000
20     18756.184082
21     18453.018544
22     35142.554101
23     22334.812330
24    480151.085585
dtype: float64

In [25]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [26]:
Eminsocrelax_df.sum()

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

In [27]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [28]:
Etripn_df.sum()

1     3.012639e+06
2     1.166828e+05
3     5.611484e+04
4     1.747803e+04
5     7.542035e+03
6     1.218573e+04
7     1.022651e+04
8     1.182157e+04
9     0.000000e+00
10    1.878932e+06
11    4.450396e+06
12    0.000000e+00
13    2.402219e+06
14    9.198751e+06
15    4.185460e+06
16    4.260775e+06
17    1.203396e+06
18    2.595200e+06
19    7.924992e+05
20    2.940490e+06
21    3.307096e+06
22    2.922840e+06
23    3.333886e+06
24    1.150694e+07
dtype: float64

In [29]:
Pex_df = ext_pyomo_vals(model.Pex)

In [30]:
Pex_df

,0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0


In [31]:
Pbio_df = ext_pyomo_vals(model.Pbio)

In [32]:
Pbio_df

,0
1,538066.6667
2,502133.3333
3,493577.7778
4,492488.8889
5,508977.7778
6,415955.5556
7,553000.0000
8,562800.0000
9,562800.0000
10,554555.5556


## Save results in csv files

In [33]:
import os 
folder = 'results_RES_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')

Pgen_df.to_csv(folder + '/Pgen.csv')
Pres_df.to_csv(folder + '/Pres.csv')
Pex_df.to_csv(folder + '/Pex.csv')